In [1]:
using Pkg 
Pkg.activate("..")

  Activating project at `~/gits/BIPs.jl`


In [2]:
using BIPs
using Statistics
using Pkg.Artifacts

In [3]:
# train_data_path = "../../DataLake/raw/train.h5"
# val_data_path = "../../DataLake/raw/val.h5"
train_data_path = "/Users/ortner/datasets/toptagging/train.h5"
val_data_path = "/Users/ortner/datasets/toptagging/val.h5"

"/Users/ortner/datasets/toptagging/val.h5"

In [4]:
train_jets, train_labels = BIPs.read_data("TQ", train_data_path)
train_labels = [reinterpret(Bool, b == 1.0) for b in train_labels]
length(train_jets)

1210997

In [7]:
val_jets, val_labels = BIPs.read_data("TQ", val_data_path)
val_labels = [reinterpret(Bool, b == 1.0) for b in val_labels]
length(val_jets)

In [5]:
train_transf_jets = data2hyp(train_jets)
println("Transformed jets: ", length(train_transf_jets))

Transformed jets: 1210997


In [ ]:
val_transf_jets = data2hyp(val_jets)
println("Transformed jets: ", length(val_transf_jets))

In [ ]:
f_bip, specs = build_ip(order=4, levels=7)
    
function bip_data(dataset_jets)
    storage = zeros(length(dataset_jets), length(specs))
    for i = 1:length(dataset_jets)
        storage[i, :] = f_bip(dataset_jets[i])
    end
    storage[:, 2:end]
end

In [ ]:
train_embedded_jets = bip_data(train_transf_jets)
println("Embedded train jets correclty")
val_embedded_jets = bip_data(val_transf_jets)
println("Embedded test jets correclty")

Now, lets fit a simple model to the data.


In [24]:
using PyCall
@pyimport sklearn.ensemble as sk_ensemble
@pyimport sklearn.metrics as sk_metrics
@pyimport sklearn.model_selection as sk_model_selection

In [31]:
GCT = sk_ensemble.HistGradientBoostingClassifier(verbose=true).fit(train_embedded_jets, train_labels)

Binning 12.843 GB of training data: 44.323 s
Binning 1.427 GB of validation data: 0.700 s
Fitting gradient boosted rounds:
[1/100] 1 tree, 31 leaves, max depth = 10, train loss: 0.62412, val loss: 0.62419, in 0.598s
[2/100] 1 tree, 31 leaves, max depth = 10, train loss: 0.56740, val loss: 0.56763, in 0.359s
[3/100] 1 tree, 31 leaves, max depth = 9, train loss: 0.51997, val loss: 0.52036, in 0.367s
[4/100] 1 tree, 31 leaves, max depth = 9, train loss: 0.47999, val loss: 0.48046, in 0.478s
[5/100] 1 tree, 31 leaves, max depth = 8, train loss: 0.44573, val loss: 0.44628, in 0.388s
[6/100] 1 tree, 31 leaves, max depth = 10, train loss: 0.41655, val loss: 0.41719, in 0.586s
[7/100] 1 tree, 31 leaves, max depth = 9, train loss: 0.39108, val loss: 0.39184, in 0.917s
[8/100] 1 tree, 31 leaves, max depth = 9, train loss: 0.36892, val loss: 0.36984, in 0.705s
[9/100] 1 tree, 31 leaves, max depth = 10, train loss: 0.34968, val loss: 0.35068, in 0.728s
[10/100] 1 tree, 31 leaves, max depth = 12, t

PyObject HistGradientBoostingClassifier(verbose=True)

# Lest test how we do performance

In [32]:
test_data_path = "../../DataLake/raw/test.h5"

"../../DataLake/raw/test.h5"

In [33]:
test_jets, test_labels = BIPs.read_data("TQ", test_data_path)
test_labels = [reinterpret(Bool, b == 1.0) for b in test_labels]
test_transf_jets = data2hyp(test_jets)
test_embedded_jets = bip_data(test_transf_jets)

403997×1473 Matrix{Float64}:
 10.9199   1.97985    -9.61555  -4.88727  …   -36.5431    -1760.17
 20.3255   0.936782  -20.1963   -2.78897       -2.14822   -7717.05
 14.4626   1.05048   -14.2088   -3.07569      -12.6807    -3013.4
 39.3236   1.2276    -39.1689   -3.65282      -48.6246   -73770.2
 12.1289   1.47541   -11.5595   -4.17361      -21.9453    -2278.93
 18.8767   1.40991   -18.437    -4.03428  …   -13.2246    -8836.07
 61.9598   1.80257   -61.7629   -5.37611     -231.468        -4.24694e5
 15.7746   1.01205   -15.582    -2.99353       -9.52968   -3828.89
 68.6222   1.86299   -68.4217   -5.561       -423.797        -5.96748e5
  7.57009  1.68356    -6.42974  -4.18674      -24.4955     -447.515
  ⋮                                       ⋱              
 28.699    1.81189   -28.2782   -5.31472      -82.1862   -40971.9
 45.7211   1.77646   -45.4996   -5.29609     -174.857        -1.67332e5
 28.4646   2.53238   -27.6481   -7.27184  …  -315.579    -53521.1
 40.4438   1.75986   -40.1546 

In [34]:
test_preds = GCT.score(test_embedded_jets, test_labels)

0.9251751869444576